# Guide to tHMM

In [1]:
import numpy as np
import scipy.stats as sp

### Synthesizing Cells (not required by the user)

In [2]:
from lineage.CellVar import CellVar as c
from lineage.CellVar import _double

In [3]:
T = np.array([[1.0, 0.0],
              [0.0, 1.0]])
    
parent_state = 1
parent_cell = c(state=parent_state, left=None, right=None, parent=None, gen=1)
left_cell, right_cell = parent_cell._divide(T)

In [4]:
print(left_cell, parent_cell.left)


 Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


In [5]:
print(right_cell, parent_cell.right)


 Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


## Creating a synthetic lineage (required by the user) "Two State Model"

In [6]:
from lineage.LineageTree import LineageTree
from lineage.StateDistribution import StateDistribution, get_experiment_time

### Creating a lineage and setting the full lineage (unpruned) as the one to be used

The required probabilities are those that define the tree and act of state switching. This process works by first creating a hidden tree of empty cells. Empty cells are those that have their states set but do not have any observations attached to them. We then draw as many observations from each state distribution and assign those observations to those cells. The $\pi$ and $T$ parameters are easy to define. The number of states is $k$. We require for $\pi$ a $k\times 1$ list of probabilities. These probabilities must add up to $1$ and they should be either in a $1$-dimensional list or a $1$-dimensional numpy array. The $T$ parameter should be a square numpy matrix of size $k\times k$. The rows are the states in which we are transitioning from and the columns are the states in which we are transitioning to. Each row of $T$ should sum to $1$. The columns need not sum to $1$.

In [7]:
# pi: the initial probability vector
pi = np.array([0.6, 0.4], dtype="float")

# T: transition probability matrix
T = np.array([[0.85, 0.15],
              [0.15, 0.85]])

The emission matrix $E$ is a little more complicated to define because this is where the user has complete freedom in defining what type of observation they care about. In particular, the user has to first begin with defining what observation he or she will want in their cells in their synthetic images. For example, if one is observing kinematics or physics, they might want to use Gaaussian distribution observations. In defining the random variables, the user will pull from a Gaussian distribution based on the mean and standard deviation of the different states he or she picks. They can also utilize the Gaussian probability distribution to define the likelihood as well. Furthermore, they can build an analytical estimator for their state distributions that yield the parameter estimates when given a list of observations. Finally, the user can also define a prune rule, which is essentially a boolean function that inspects a cell's observations and returns True if the cell's subtree (all the cells that are related to the cell in question and are of older generation) is to be pruned or False if the cell is safe from pruning. In the Gaussian example, a user can remove a cell's subtree if its observation is higher or lower than some fixed value.

We have already built, as an example, and as bioengineers, a model that resembles lineage trees. In our synthetic model, our emissions are multivariate. This first emission is a Bernoulli observation, $0$ implying death and $1$ implying division. The second and third emissions are continuous and are from exponential and gamma distributions respectively. Though these can be thought of cell lifetime's or periods in a certain cell phase, we want the user to know that these values can really mean anything and they are completely free in choosing what the emissions and their values mean. We define ways to calculate random variables for these multivariate observations and likelihoods of an observations. We also provide as a prune rule, keeping with the cell analogy, that if a cell has a $0$ in its Bernoulli observation, then its subtree is pruned from the full lineage tree. Though this will obviously introduce bias into estimation, we keep both the full tree and the pruned tree in the lineage objects, in the case a user would like to see the effects of analyzing on one versus the other.

Ultimately, $E$ is defined as a $k\times 1$ size list of state distribution objects. These distribution objects are rich in what they can already do, and a user can easily add more to their functionality. They only need to be instantiated by what parameters define that state's distribution.

In [8]:
# E: states are defined as StateDistribution objects

# State 0 parameters "Resistant"
state0 = 0
bern_p0 = 0.99
gamma_loc = 0
gamma_a0 = 20
gamma_scale0 = 5

# State 1 parameters "Susceptible"
state1 = 1
bern_p1 = 0.88
gamma_a1 = 10
gamma_scale1 = 1

state_obj0 = StateDistribution(state0, bern_p0, gamma_a0, gamma_loc, gamma_scale0)
state_obj1 = StateDistribution(state1, bern_p1, gamma_a1, gamma_loc,  gamma_scale1)

E = [state_obj0, state_obj1]

The final required parameters are more obvious. The first is the desired number of cells one would like in their full unpruned lineage tree. This can be any number. The lineage tree is built 'from left to right'. What this means is that, we construct the binary tree by going to the left-most cell, dividing then walking through the generation. For example, if someone requested for

In [9]:
desired_num_cells = 2**7 - 1 
prune_boolean = False # To get the full tree

In [10]:
lineage1 = LineageTree(pi, T, E, desired_num_cells, prune_boolean)
print(lineage1)

This tree is NOT pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 46 cells of state 0, 
 	 There are 81 cells of state 1.
 This UNpruned tree has 127 cells in total


### Obtaining how long the experiment ran by checking the time length of the longest branch

In [11]:
longest_branch_time = get_experiment_time(lineage1)
print(longest_branch_time)

640.8796709918277


### Estimation of distribution parameters using our estimators for full lineage

In [12]:
for state in range(lineage1.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage1.lineage_stats[state].full_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
                    estimated state State object w/ parameters: 0.9782608695631381, 19.136048556906317, 0.0, 5.578969577237897.
original parameters given for state State object w/ parameters: 0.99, 20, 0, 5.
State 1:
                    estimated state State object w/ parameters: 0.8518518518509831, 8.995820934805481, 0.0, 1.0995343388294743.
original parameters given for state State object w/ parameters: 0.88, 10, 0, 1.


### Estimation of distribution parameters using our estimators for pruned lineage

In [13]:
for state in range(lineage1.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage1.lineage_stats[state].pruned_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
                    estimated state State object w/ parameters: 0.9749999999976251, 18.845161211369827, 0.0, 5.618459574964954.
original parameters given for state State object w/ parameters: 0.99, 20, 0, 5.
State 1:
                    estimated state State object w/ parameters: 0.8688524590151842, 8.655081586076848, 0.0, 1.1322523354400162.
original parameters given for state State object w/ parameters: 0.88, 10, 0, 1.


### Analyzing our first full lineage

In [14]:
from lineage.Analyze import Analyze, accuracy

X = [lineage1] # population just contains one lineage
states = [cell.state for cell in lineage1.output_lineage]
print(states)
deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 2) # find two states


[1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]


In [15]:
tHMMobj.estimate.E

[State object w/ parameters: 0.9999999999985294, 9.870477855699972, 0.0, 1.0051165725688513.,
 State object w/ parameters: 0.7796610169482047, 1.8743457152107759, 0.0, 45.55257852470821.]

### Estimated Markov parameters ($\pi$, $T$, $E$)

In [16]:
print(tHMMobj.estimate.pi)

[1.00000000e+00 1.71649713e-21]


In [17]:
print(tHMMobj.estimate.T)

[[0.73681081 0.26318919]
 [0.17102326 0.82897674]]


In [18]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state])

State object w/ parameters: 0.9999999999985294, 9.870477855699972, 0.0, 1.0051165725688513.
State object w/ parameters: 0.7796610169482047, 1.8743457152107759, 0.0, 45.55257852470821.


## Trying another lineage, this time pruning branches with ancestors that die

In [19]:
desired_num_cells = 2**12 -1 
prune_boolean = True # To get pruned tree

In [20]:
lineage2 = LineageTree(pi, T, E, desired_num_cells, prune_boolean)
print(lineage2)

This tree is pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 1640 cells of state 0, 
 	 There are 1147 cells of state 1.
 This pruned tree has 2787 cells in total


In [21]:
longest2 = get_experiment_time(lineage2)
print(longest2)

1382.82126174806


### Estimation of distribution parameters using our estimators for pruned lineage

In [22]:
for state in range(lineage2.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage2.lineage_stats[state].pruned_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
                    estimated state State object w/ parameters: 0.9884146341462818, 20.58891050486496, 0.0, 4.83520476700565.
original parameters given for state State object w/ parameters: 0.99, 20, 0, 5.
State 1:
                    estimated state State object w/ parameters: 0.8700959023539023, 10, 0.0, 1.
original parameters given for state State object w/ parameters: 0.88, 10, 0, 1.


### Analyzing a population of lineages

In [23]:
X = [lineage1, lineage2] # population just contains one lineage

deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 2) # find two states

In [24]:
from lineage.Analyze import accuracy
accuracy(tHMMobj, all_states)

[99.21259842519686, 99.85647649802655]

### Estimated Markov parameters ($\pi$, $T$, $E$)

In [25]:
print(tHMMobj.estimate.pi)

[1.00000000e+000 4.65632526e-167]


In [26]:
print(tHMMobj.estimate.T)

[[0.84690368 0.15309632]
 [0.15547017 0.84452983]]


In [27]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state])

State object w/ parameters: 0.9881446354474821, 19.979474096503303, 0.0, 4.989859729801417.
State object w/ parameters: 0.8687856560716595, 9.918174841943593, 0.0, 1.0057428637580734.


## Creating a synthetic lineage that has three states

Here we generate a lineage with three states, which would be 1) Susciptible 2) Middle State 3) Resistant. The aim here is to show the transition from susciptible to resistant state doesn't happen immediately, and there is a gradual transition which is modeled as a middle state. The point to be considered here is that transition from 1 to 3 or otherwise is not possible so the probability of these transitions are zero, and most likely the initial cells are in susciptible state.

**State 1**: Susceptible

**State 2**: Transition state

**State 3**: Resistant state


In [28]:
# pi: the initial probability vector
pi_3 = np.array([0.5, 0.25, 0.25])

# T: transition probability matrix
T_3 = np.array([[0.65, 0.35, 0.00],
                [0.20, 0.40, 0.40],
                [0.00, 0.10, 0.90]])

In [29]:
# E: states are defined as StateDistribution objects

# State 0 parameters "Susciptible"
state0 = 0
bern_p0 = 0.7
gamma_a0 = 5.0
gamma_scale0 = 1.0

# State 1 parameters "Middle state"
state1 = 1
bern_p1 = 0.85
gamma_a1 = 10.0
gamma_scale1 = 2.0

# State 2 parameters "Resistant"
state2 = 2
bern_p2 = 0.99
gamma_a2 = 15.0
gamma_scale2 = 3.0

state_obj0 = StateDistribution(state0, bern_p0, gamma_a0, gamma_loc, gamma_scale0)
state_obj1 = StateDistribution(state1, bern_p1, gamma_a1, gamma_loc, gamma_scale1)
state_obj2 = StateDistribution(state2, bern_p2, gamma_a2, gamma_loc, gamma_scale2)

E_3 = [state_obj0, state_obj1, state_obj2]

In [30]:
desired_num_cells = 2**13 - 1 
prune_boolean = False # To get the full tree

In [31]:
lineage3 = LineageTree(pi_3, T_3, E_3, desired_num_cells, prune_boolean)
print(lineage3)

This tree is NOT pruned. It is made of 3 states.
 For each state in this tree: 
 	 There are 634 cells of state 0, 
 	 There are 1343 cells of state 1, 
 	 There are 6214 cells of state 2.
 This UNpruned tree has 8191 cells in total


In [32]:
longest3 = get_experiment_time(lineage3)
print(longest3)

672.9443474699947


### Estimation of distribution parameters using our estimators for full lineage (3 state)

In [33]:
for state in range(lineage3.num_states):
    print("State {}:".format(state))
    print("estimated state", E_3[state].estimator(lineage3.lineage_stats[state].full_lin_cells_obs))
    print("estimated state", E_3[state].estimator(lineage3.lineage_stats[state].pruned_lin_cells_obs))
    print("true_____ state", E_3[state])

State 0:
estimated state State object w/ parameters: 0.7018927444794316, 4.9177053184498005, 0.0, 1.0225911241379007.
estimated state State object w/ parameters: 0.7057142857141682, 10, 0.0, 1.
true_____ state State object w/ parameters: 0.7, 5.0, 0, 1.0.
State 1:
estimated state State object w/ parameters: 0.8518242740133504, 10.01393844551825, 0.0, 1.9917946103906345.
estimated state State object w/ parameters: 0.8473581213306561, 10.208045296332546, 0.0, 1.9496585125873929.
true_____ state State object w/ parameters: 0.85, 10.0, 0, 2.0.
State 2:
estimated state State object w/ parameters: 0.9890569681364503, 15.157838177093538, 0.0, 2.9640907340594618.
estimated state State object w/ parameters: 0.9895212285456011, 15.163746076709831, 0.0, 2.962885322767819.
true_____ state State object w/ parameters: 0.99, 15.0, 0, 3.0.


### Analyzing a three state lineage

In [34]:
X = [lineage3] # population just contains one lineage

deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 3) # find three states

In [35]:
accuracy(tHMMobj, all_states)

[2.4172872665120257]

### Estimated Markov parameters ($\pi$, $T$, $E$)

In [36]:
print(tHMMobj.estimate.pi)

[1.00000000e+000 1.25295758e-268 8.84016955e-074]


In [37]:
print(tHMMobj.estimate.T)

[[0.93477959 0.01389879 0.05132162]
 [0.16454364 0.19320554 0.64225083]
 [0.23726475 0.15931921 0.60341604]]
